In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1. Install required libraries**

In [2]:
%%capture
!pip install -q ultralytics opencv-python-headless tqdm matplotlib

# **2. Imports & Configuration**

In [7]:
import os, cv2, glob, yaml, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO
from IPython.display import FileLink, display


class Config:
    data_dir   = "/kaggle/input/welding-defect-object-detection/The Welding Defect Dataset/The Welding Defect Dataset"
    yaml_path  = "data.yaml"
    epochs     = 50
    imgsz      = 640
    batch      = 16
    max_frames = 30          # images for the video
    fps        = 4
    conf_thr   = 0.20
    iou_thr    = 0.45

cfg = Config()

# **3. Create dataset YAML**

In [4]:
def create_yaml():
    yaml_dict = {
        'train': str(Path(cfg.data_dir) / 'train' / 'images'),
        'val'  : str(Path(cfg.data_dir) / 'valid' / 'images'),   # <-- correct folder
        'nc'   : 4,
        'names': ['crack', 'porosity', 'slagging', 'undercut']
    }
    with open(cfg.yaml_path, 'w') as f:
        yaml.dump(yaml_dict, f)
    print(f"data.yaml created → {cfg.yaml_path}")

create_yaml()

data.yaml created → data.yaml


# **4. Train YOLOv8n**

In [5]:
print("Training YOLOv8n …")
model = YOLO('yolov8n.pt')                     # pretrained nano model

results = model.train(
    data=cfg.yaml_path,
    epochs=cfg.epochs,
    imgsz=cfg.imgsz,
    batch=cfg.batch,
    device=0,                                 # GPU
    name='weld_defect_yolov8',
    patience=10,
    save=True,
    # optional augmentation for better results
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    flipud=0.5, fliplr=0.5
)

# best checkpoint
best_model_path = max(
    glob.glob("runs/detect/weld_defect_yolov8*/weights/best.pt"),
    key=os.path.getctime
)
print(f"Best model: {best_model_path}")

Training YOLOv8n …
Ultralytics 8.3.225 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=weld_defect_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=Tru

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        176        573      0.545      0.627       0.58      0.347
                 crack         87        127      0.504      0.772      0.639      0.404
              porosity        119        253      0.634      0.763       0.75      0.484
              slagging         90        193      0.498      0.347      0.351      0.154
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /kaggle/working/runs/detect/weld_defect_yolov8
Best model: runs/detect/weld_defect_yolov8/weights/best.pt


# **5. Video generation + auto-download**

In [8]:
def create_detection_video():
    print("\nGenerating detection video …")
    model = YOLO(best_model_path)

    # ---- validation images (valid folder) ----
    val_img_paths = glob.glob(os.path.join(cfg.data_dir, "valid", "images", "*.*"))
    if not val_img_paths:
        print("No images found in valid/images!")
        return None

    val_images = sorted(val_img_paths)[:cfg.max_frames]
    print(f"Using {len(val_images)} validation images")

    # ---- safe draw function (no matplotlib warning) ----
    def draw_detections(img_path):
        try:
            img = cv2.imread(img_path)
            if img is None:
                img = np.random.randint(100, 200, (640, 640, 3), dtype=np.uint8)
            else:
                img = cv2.resize(img, (640, 640))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            res = model.predict(
                source=img_path,
                conf=cfg.conf_thr,
                iou=cfg.iou_thr,
                imgsz=cfg.imgsz,
                verbose=False
            )[0]

            if res.boxes is not None and len(res.boxes) > 0:
                for box in res.boxes.data.cpu().numpy():
                    x1, y1, x2, y2, conf, cls = box
                    cls = int(cls)
                    if cls >= len(model.names):  # safety
                        cls = 0
                    if conf < cfg.conf_thr:
                        continue

                    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                    if x1 >= x2 or y1 >= y2:
                        continue

                    label = f"{model.names[cls]} {conf:.2f}"
                    color = tuple(int(c*255) for c in plt.cm.Set1(cls % 10)[:3])

                    cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 2)
                    tw, th = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    cv2.rectangle(img_rgb, (x1, y1-th-10), (x1+tw, y1), color, -1)
                    cv2.putText(img_rgb, label, (x1, y1-5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            else:
                cv2.putText(img_rgb, "No Defect", (50, 80),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
            return img_rgb

        except Exception as e:
            print(f"Error on {img_path}: {e}")
            err = np.zeros((640,640,3), dtype=np.uint8)
            cv2.putText(err, "ERROR", (200,320), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 4)
            return err

    # ---- create frames ----
    frames = [draw_detections(p) for p in tqdm(val_images, desc="Frames")]

    # ---- write video ----
    out_path = "/kaggle/working/weld_defect_detection.mp4"
    h, w = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(out_path, fourcc, cfg.fps, (w, h))

    print("Writing video …")
    for fr in tqdm(frames, desc="Video"):
        writer.write(cv2.cvtColor(fr, cv2.COLOR_RGB2BGR))
    writer.release()

    size_mb = os.path.getsize(out_path) / 1e6
    print(f"Video created: {out_path} ({size_mb:.1f} MB)")

    # ---- auto-download ----
    display(FileLink(out_path))
    from google.colab import files
    files.download(out_path)
    return out_path

# Run
video_path = create_detection_video()


Generating detection video …
Using 30 validation images


Frames: 100%|██████████| 30/30 [00:00<00:00, 56.93it/s]


Writing video …


Video: 100%|██████████| 30/30 [00:00<00:00, 218.56it/s]

Video created: /kaggle/working/weld_defect_detection.mp4 (1.9 MB)


/kaggle/working/weld_defect_detection.mp4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>